In [8]:
import torch
from datasets import load_dataset
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments
)
from trl import SFTTrainer
from evaluate import load
import time

# Device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load the dataset
dataset = load_dataset("argilla/ifeval-like-data", "filtered")

# Model name
model_name = "goat_ifeval_dataset/checkpoint-1500"

# Load the tokenizer for Mistral
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    add_eos_token=True,      # Add end-of-sequence token to the tokenizer
    use_fast=True,           # Use the fast tokenizer implementation
    padding_side='left'      # Pad sequences on the left side
)
tokenizer.pad_token = tokenizer.eos_token  # Set padding token to EOS token


In [9]:
from peft import get_peft_model 
# Quantization configuration using bitsandbytes library
compute_dtype = getattr(torch, "bfloat16")  # Set computation data type to bfloat16
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,                    # Enable loading the model in 4-bit precision
    bnb_4bit_quant_type="nf4",            # Specify quantization type as Normal Float 4
    bnb_4bit_compute_dtype=compute_dtype, # Set computation data type
    bnb_4bit_use_double_quant=True,       # Use double quantization for better accuracy
)

# Load the pre-trained model with the specified quantization configuration
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,  # Apply quantization configuration
    device_map="auto"                # Automatically map layers to devices
)

# Prepare the model for k-bit (e.g., 4-bit) training
model = prepare_model_for_kbit_training(model)

# lora
# Low-Rank Adaptation (LoRA) configuration for efficient fine-tuning
lora_config = LoraConfig(
    lora_alpha=16,             # Scaling factor for LoRA updates
    lora_dropout=0.05,         # Dropout rate applied to LoRA layers
    r=5,                      # Rank of the LoRA decomposition
    bias="none",               # No bias is added to the LoRA layers
    task_type="CAUSAL_LM",     # Specify the task as causal language modeling
    target_modules=[           # Modules to apply LoRA to
        'k_proj', 'q_proj', 'v_proj', 'o_proj',
        'gate_proj', 'down_proj', 'up_proj'
    ]
)

model = get_peft_model(model, lora_config)



model.config.pad_token_id = tokenizer.pad_token_id  # Set the model's padding token ID

# print how much does the model occupy in memory
print(f"Model size: {sum(p.numel() for p in model.parameters())}")

/home/usuario/.local/lib/python3.10/site-packages/accelerate/utils/modeling.py:1390: UserWarning: Current model requires 1024.0 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `load_in_8bit_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 

In [ ]:
from datasets import DatasetDict, Dataset
#  Paso 1: Crear la columna 'conversations' en el dataset
def create_conversations(split_dataset):
    conversations = []
    for example in split_dataset:
        # print(example)  # Uncomment for debugging
        conversation = [example['prompt'], example['response']]
        conversations.append({'conversations': conversation})
    new_dataset = Dataset.from_list(conversations)
    return new_dataset

split_dataset = dataset['train'].train_test_split(test_size=0.15, seed=42)
train_dataset = split_dataset['train']
test_set = split_dataset['test']
# divide el valid en 75 y 25
split_train = train_dataset.train_test_split(test_size=0.10, seed=42)
train_dataset = split_train['train']
valid_dataset = split_train['test']

new_dataset = DatasetDict({
    'train': create_conversations(train_dataset),   
    'validation': create_conversations(valid_dataset),
})


In [ ]:
def format_conversation(examples):
    # Unir las conversaciones en un solo string

    joined_conversations = ["\n".join(conv) if isinstance(conv, list) else conv for conv in examples['conversations']]
    
    # Tokenizar las conversaciones unidas
    tokenized = tokenizer(
        joined_conversations,
        truncation=True,
        max_length=512,
        padding="max_length",
        return_tensors="pt"
    )
    # Convertir tensores a listas para evitar problemas
    tokenized = {k: v.tolist() for k, v in tokenized.items()}
    return tokenized

# Aplicar la tokenización al dataset
tokenized_dataset = new_dataset.map(format_conversation, batched=True)
tokenized_dataset



Map:   0%|          | 0/43099 [00:00<?, ? examples/s]

Map:   0%|          | 0/4789 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['conversations', 'input_ids', 'attention_mask'],
        num_rows: 43099
    })
    validation: Dataset({
        features: ['conversations', 'input_ids', 'attention_mask'],
        num_rows: 4789
    })
})

In [ ]:
# Paso 3: Configurar el formato del dataset para PyTorch
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])

len(tokenized_dataset["train"][0]["input_ids"])

512

In [12]:
import time
import torch
from transformers import PreTrainedModel, PreTrainedTokenizer
from datasets import Dataset

class PerformanceBenchmark:
    """
    A class to benchmark the performance of a model on a given dataset.
    
    Attributes:
    -----------
    model : transformers.PreTrainedModel
        The model to be benchmarked.
    tokenizer : transformers.PreTrainedTokenizer
        The tokenizer associated with the model.
    dataset : datasets.Dataset
        The dataset on which the model's performance will be evaluated.
    """
    
    def __init__(self, model: PreTrainedModel, tokenizer: PreTrainedTokenizer, dataset: Dataset):
        """
        Initializes the PerformanceBenchmark with the provided model, tokenizer, and dataset.
        
        Parameters:
        -----------
        model : transformers.PreTrainedModel
            The model to be benchmarked.
        tokenizer : transformers.PreTrainedTokenizer
            The tokenizer for encoding the inputs for the model.
        dataset : datasets.Dataset
            The dataset on which the model's performance will be evaluated.
        """
        self.model = model
        self.tokenizer = tokenizer
        self.dataset = dataset

    def compute_parameters(self):
        """
        Computes the total number of parameters and the number of trainable parameters.
        
        Returns:
        --------
        dict :
            A dictionary containing:
            - `total_params`: The total number of parameters in the model.
            - `trainable_params`: The number of trainable parameters in the model.
        """
        total_params = sum(p.numel() for p in self.model.parameters())  # Total parameters
        trainable_params = sum(p.numel() for p in self.model.parameters() if p.requires_grad)  # Trainable parameters
        
        return {
            "total_params": total_params,
            "trainable_params": trainable_params
        }

    def compute_size(self):
        """
        Computes the size of the model in terms of the number of parameters 
        and memory usage in megabytes (MB).

        Returns:
        --------
        dict :
            A dictionary containing the number of parameters (`num_params`) and 
            the model size in MB (`model_size_mb`).
        """
        num_params = sum(p.numel() for p in self.model.parameters())
        model_size_mb = sum(p.element_size() * p.nelement() for p in self.model.parameters()) / (1024**2)
        
        return {"num_params": num_params, "model_size_mb": model_size_mb}

    def time_pipeline(self):
        """
        Measures the total time and average time taken by the model to process 
        the dataset.
        
        This method will use the tokenizer to encode the inputs before passing them 
        to the model.

        Returns:
        --------
        dict :
            A dictionary containing the total processing time in seconds (`total_time_sec`) 
            and the average time per example (`avg_time_per_example_sec`).
        """
        start_time = time.time()
        
        for example in self.dataset:
            inputs = example['prompt']
            
            # Verifica si 'inputs' es una lista y, de ser así, conviértela en una cadena
            if isinstance(inputs, list):
                inputs = " ".join(inputs)
            
            # Tokeniza la entrada con padding y truncation habilitados
            tokenized_input = self.tokenizer(
                inputs, 
                padding=True, 
                truncation=True, 
                return_tensors="pt",
                max_length=512  # Ajusta según las necesidades de tu modelo
            ).to(self.model.device)
            
            _ = self.model.generate(**tokenized_input, max_new_tokens=10)
        
        end_time = time.time()
        total_time = end_time - start_time
        avg_time_per_example = total_time / len(self.dataset) if len(self.dataset) > 0 else float('inf')
        
        return {"total_time_sec": total_time, "avg_time_per_example_sec": avg_time_per_example}

    def compute_latency(self):
        """
        Computes the average latency of the model, defined as the time taken 
        to process a single example from the dataset.

        Returns:
        --------
        dict :
            A dictionary containing the average latency in seconds (`avg_latency_sec`).
        """
        latencies = []
        
        for example in self.dataset:
            inputs = example['prompt']
            # Verifica si 'inputs' es una lista y, de ser así, conviértela en una cadena
            if isinstance(inputs, list):
                inputs = " ".join(inputs)
            
            # Tokeniza la entrada con padding y truncation habilitados
            tokenized_input = self.tokenizer(
                inputs, 
                padding=True, 
                truncation=True, 
                return_tensors="pt",
                max_length=512  # Ajusta según las necesidades de tu modelo
            ).to(self.model.device)
            
            start_time = time.time()
            _ = self.model.generate(**tokenized_input, max_new_tokens=10)
            end_time = time.time()
            
            latencies.append(end_time - start_time)
        
        avg_latency = sum(latencies) / len(latencies) if len(latencies) > 0 else float('inf')
        return {"avg_latency_sec": avg_latency}

    def compute_throughput(self):
        """
        Computes the throughput of the model, defined as the number of examples 
        processed per second.

        Returns:
        --------
        dict :
            A dictionary containing the throughput in examples per second (`throughput_examples_per_sec`).
        """
        start_time = time.time()
        
        for example in self.dataset:
            inputs = example['prompt']
            
            # Verifica si 'inputs' es una lista y, de ser así, conviértela en una cadena
            if isinstance(inputs, list):
                inputs = " ".join(inputs)
            
            # Tokeniza la entrada con padding y truncation habilitados
            tokenized_input = self.tokenizer(
                inputs, 
                padding=True, 
                truncation=True, 
                return_tensors="pt",
                max_length=512  # Ajusta según las necesidades de tu modelo
            ).to(self.model.device)
            
            _ = self.model.generate(**tokenized_input, max_new_tokens=10)
        
        end_time = time.time()
        total_time = end_time - start_time
        throughput = len(self.dataset) / total_time if total_time > 0 else 0
        
        return {"throughput_examples_per_sec": throughput}
    

    def run_benchmark(self):
        """
        Runs all the benchmark metrics (size, time, latency, throughput, and FLOPs) 
        and returns the results.

        Returns:
        --------
        dict :
            A dictionary containing all the computed metrics for the model. 
            Includes size, parameters, time, latency, throughput, and FLOPs estimates.
        """
        metrics = {}
        metrics['Size'] = self.compute_size()
        metrics['Parameters'] = self.compute_parameters()
        metrics['Time'] = self.time_pipeline()
        metrics['Latency'] = self.compute_latency()
        metrics['Throughput'] = self.compute_throughput()
        return metrics

# Deshabilitar gradientes para ahorrar memoria y cómputo
model.eval()
torch.set_grad_enabled(False)  # Disable gradient computation globally
accuracy_score = load("accuracy")

print(test_set)
# limit test set to 10 examples
test_set = test_set.select(range(10))
# Instanciar la clase PerformanceBenchmark con el modelo, tokenizer y dataset de prueba
benchmark = PerformanceBenchmark(model, tokenizer, test_set)
# Ejecutar el benchmark para calcular las métricas de rendimiento
results = benchmark.run_benchmark()

# Mostrar los resultados del benchmark
print(results)


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Dataset({
    features: ['key', 'prompt', 'response', 'instruction_id_list', 'kwargs', 'prompt_level_strict_acc', 'inst_level_strict_acc', 'prompt_level_loose_acc', 'inst_level_loose_acc'],
    num_rows: 8451
})


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Settin

{'Size': {'num_params': 3771469824, 'model_size_mb': 4403.015625}, 'Parameters': {'total_params': 3771469824, 'trainable_params': 13107200}, 'Time': {'total_time_sec': 9.358814001083374, 'avg_time_per_example_sec': 0.9358814001083374}, 'Latency': {'avg_latency_sec': 0.9031650304794312}, 'Throughput': {'throughput_examples_per_sec': 1.1536195807807155}}
